In [26]:
# add libraries
import requests
import json
import pandas as pd
import os
import pprint

#spotipy
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Data Acquisition
### Grab data from different API sources (Spotify, Genius, Billboard 100)

In [27]:
# function for grabbing api key
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

#### Spotify

In [28]:
spotify_client = "../spotify_client"
spotify_client_secret = "../spotify_client_secret"
spotify_redirect = "../spotify_redirect"

os.environ['SPOTIPY_CLIENT_ID'] = get_file_contents(spotify_client)
os.environ['SPOTIPY_CLIENT_SECRET'] = get_file_contents(spotify_client_secret)
os.environ['SPOTIPY_REDIRECT_URI'] = get_file_contents(spotify_redirect)


spotify_artist_id = "06HL4z0CvFAxyc27GXpf02"

Endpoint usage: artist albums -> for each get album tracks -> for each track get the tracks audio features

In [29]:
# # artist albums
# scope = "user-library-read"

# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

# results = sp.current_user_saved_tracks()
# for idx, item in enumerate(results['items']):
#     track = item['track']
#     print(idx, track['artists'][0]['name'], " – ", track['name'])

In [30]:
scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [31]:
taylor_url = f'spotify:artist:{spotify_artist_id}'
results = sp.artist_albums(taylor_url, album_type='album', country = 'CA')

albums = results['items']

while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

# for album in albums:
#     print(album['id'])

for album in albums:
    print(album['name'])

Midnights (The Til Dawn Edition)
Midnights (The Til Dawn Edition)
Midnights (3am Edition)
Midnights (3am Edition)
Midnights
Midnights
Red (Taylor's Version)
Red (Taylor's Version)
Fearless (Taylor's Version)
evermore (deluxe version)
evermore (deluxe version)
evermore
evermore
folklore: the long pond studio sessions (from the Disney+ special) [deluxe edition]
folklore: the long pond studio sessions (from the Disney+ special) [deluxe edition]
folklore (deluxe version)
folklore (deluxe version)
folklore
folklore
Lover
Taylor Swift Karaoke: reputation
reputation
reputation (Big Machine Radio Release Special)
reputation Stadium Tour Surprise Song Playlist
1989
1989 (Big Machine Radio Release Special)
Taylor Swift Karaoke: 1989 (Deluxe Edition)
Taylor Swift Karaoke: 1989
1989 (Deluxe Edition)
1989 (Deluxe Edition)
Red
Red (Deluxe Edition)
Red (Big Machine Radio Release Special)
Taylor Swift Karaoke: Red
Speak Now World Tour Live
Speak Now
Speak Now (Big Machine Radio Release Special)
Taylor

In [32]:
# there are some duplicate albums because some songs are explicit creating a new album
# so lets just try to take the unique album names
# after grabbing unique album also take the singles potentially
# lets take songs from these albums
# pick Taylor's Version and Deluxe Albums
# deluxe version usually is a rerelease but with more songs and with some acoustic versions

# I will put the index of the album beside the name
# Midnights (3am Edition)        0 
# Red (Taylor's Version)         4
# Fearless (Taylor's Version)    6
# evermore (deluxe version)      7
# folklore (deluxe version)     13
# Lover                         17
# reputation                    19
# 1989 (deluxe version)         26               
# Taylor Swift                  45
# Speak Now (Deluxe Edition)    36
picked_albums= []

picked_albums.append(albums[2])
picked_albums.append(albums[6])
picked_albums.append(albums[8])
picked_albums.append(albums[9])
picked_albums.append(albums[15])
picked_albums.append(albums[19])
picked_albums.append(albums[21])
picked_albums.append(albums[28])
picked_albums.append(albums[45])
picked_albums.append(albums[38])

In [33]:
for album in picked_albums:
    print(album['name'])

Midnights (3am Edition)
Red (Taylor's Version)
Fearless (Taylor's Version)
evermore (deluxe version)
folklore (deluxe version)
Lover
reputation
1989 (Deluxe Edition)
Taylor Swift
Speak Now (Deluxe Edition)


In [49]:
tracks = []
album_names = []
album_release_date = []
for album in picked_albums:
    for i in range(album['total_tracks']):
        album_names.append(album['name'])
        album_release_date.append(album['release_date'])
    results = sp.album_tracks(album['id'])
    tracks.extend(results['items'])

names_tracks = []
for track in tracks:
    names_tracks.append(track['name'])
    print(track['name'])

print(len(tracks))
# there are 197 tracks

Lavender Haze
Maroon
Anti-Hero
Snow On The Beach (feat. Lana Del Rey)
You're On Your Own, Kid
Midnight Rain
Question...?
Vigilante Shit
Bejeweled
Labyrinth
Karma
Sweet Nothing
Mastermind
The Great War
Bigger Than The Whole Sky
Paris
High Infidelity
Glitch
Would've, Could've, Should've
Dear Reader
State Of Grace (Taylor's Version)
Red (Taylor's Version)
Treacherous (Taylor's Version)
I Knew You Were Trouble (Taylor's Version)
All Too Well (Taylor's Version)
22 (Taylor's Version)
I Almost Do (Taylor's Version)
We Are Never Ever Getting Back Together (Taylor's Version)
Stay Stay Stay (Taylor's Version)
The Last Time (feat. Gary Lightbody of Snow Patrol) (Taylor’s Version)
Holy Ground (Taylor's Version)
Sad Beautiful Tragic (Taylor's Version)
The Lucky One (Taylor's Version)
Everything Has Changed (feat. Ed Sheeran) (Taylor’s Version)
Starlight (Taylor's Version)
Begin Again (Taylor's Version)
The Moment I Knew (Taylor's Version)
Come Back...Be Here (Taylor's Version)
Girl At Home (Taylor'

In [50]:
# using the track id get track audio features and audio analysis
# getting audio features requires sending a get request for a comma seperated list of spotify ids with max 100 IDS

track_ids = []

for track in tracks:
    track_ids.append(track['id'])

In [51]:
# now we make api call to get the audio features
audio_features = []

results = sp.audio_features(track_ids[0:100])
audio_features.extend(results)

results = sp.audio_features(track_ids[100:])
audio_features.extend(results)

In [52]:
audio_features

[{'danceability': 0.735,
  'energy': 0.444,
  'key': 10,
  'loudness': -10.519,
  'mode': 1,
  'speechiness': 0.0684,
  'acousticness': 0.204,
  'instrumentalness': 0.0012,
  'liveness': 0.17,
  'valence': 0.0984,
  'tempo': 97.038,
  'type': 'audio_features',
  'id': '4g2c7NoTWAOSYDy44l9nub',
  'uri': 'spotify:track:4g2c7NoTWAOSYDy44l9nub',
  'track_href': 'https://api.spotify.com/v1/tracks/4g2c7NoTWAOSYDy44l9nub',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4g2c7NoTWAOSYDy44l9nub',
  'duration_ms': 202396,
  'time_signature': 4},
 {'danceability': 0.658,
  'energy': 0.378,
  'key': 7,
  'loudness': -8.3,
  'mode': 1,
  'speechiness': 0.0379,
  'acousticness': 0.0593,
  'instrumentalness': 0,
  'liveness': 0.0976,
  'valence': 0.0382,
  'tempo': 108.034,
  'type': 'audio_features',
  'id': '199E1RRrVmVTQqBXih5qRC',
  'uri': 'spotify:track:199E1RRrVmVTQqBXih5qRC',
  'track_href': 'https://api.spotify.com/v1/tracks/199E1RRrVmVTQqBXih5qRC',
  'analysis_url': 'https://api

In [53]:
# make a data frame to store all the data
first_df = pd.DataFrame.from_dict(audio_features)

In [54]:
first_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.735,0.444,10,-10.519,1,0.0684,0.20400,0.001200,0.1700,0.0984,97.038,audio_features,4g2c7NoTWAOSYDy44l9nub,spotify:track:4g2c7NoTWAOSYDy44l9nub,https://api.spotify.com/v1/tracks/4g2c7NoTWAOS...,https://api.spotify.com/v1/audio-analysis/4g2c...,202396,4
1,0.658,0.378,7,-8.300,1,0.0379,0.05930,0.000000,0.0976,0.0382,108.034,audio_features,199E1RRrVmVTQqBXih5qRC,spotify:track:199E1RRrVmVTQqBXih5qRC,https://api.spotify.com/v1/tracks/199E1RRrVmVT...,https://api.spotify.com/v1/audio-analysis/199E...,218271,4
2,0.638,0.634,4,-6.582,1,0.0457,0.13300,0.000001,0.1520,0.5190,96.953,audio_features,02Zkkf2zMkwRGQjZ7T4p8f,spotify:track:02Zkkf2zMkwRGQjZ7T4p8f,https://api.spotify.com/v1/tracks/02Zkkf2zMkwR...,https://api.spotify.com/v1/audio-analysis/02Zk...,200690,4
3,0.659,0.323,9,-13.425,1,0.0436,0.73500,0.003210,0.1160,0.1540,110.007,audio_features,6ADDIJxxqzM9LMpm78yzQG,spotify:track:6ADDIJxxqzM9LMpm78yzQG,https://api.spotify.com/v1/tracks/6ADDIJxxqzM9...,https://api.spotify.com/v1/audio-analysis/6ADD...,256124,4
4,0.694,0.380,2,-10.307,1,0.0614,0.41600,0.000008,0.1260,0.3760,120.044,audio_features,7gVWKBcfIW93YxNBi3ApIE,spotify:track:7gVWKBcfIW93YxNBi3ApIE,https://api.spotify.com/v1/tracks/7gVWKBcfIW93...,https://api.spotify.com/v1/audio-analysis/7gVW...,194207,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,0.511,0.719,4,-3.457,1,0.0265,0.16600,0.000005,0.3400,0.2890,147.852,audio_features,0vvt4IZOMkRug195S4MUq0,spotify:track:0vvt4IZOMkRug195S4MUq0,https://api.spotify.com/v1/tracks/0vvt4IZOMkRu...,https://api.spotify.com/v1/audio-analysis/0vvt...,234547,4
193,0.582,0.765,7,-3.648,1,0.0324,0.02680,0.000002,0.1030,0.5590,131.982,audio_features,2R7C9dDqv1UPycvepBFziA,spotify:track:2R7C9dDqv1UPycvepBFziA,https://api.spotify.com/v1/tracks/2R7C9dDqv1UP...,https://api.spotify.com/v1/audio-analysis/2R7C...,275960,4
194,0.541,0.453,2,-6.459,1,0.0266,0.74800,0.000000,0.2000,0.3770,141.757,audio_features,6IH2Z54gYPLqgJtxZjX80i,spotify:track:6IH2Z54gYPLqgJtxZjX80i,https://api.spotify.com/v1/tracks/6IH2Z54gYPLq...,https://api.spotify.com/v1/audio-analysis/6IH2...,292560,4
195,0.563,0.443,5,-5.078,1,0.0247,0.84200,0.000000,0.2930,0.3130,80.962,audio_features,62rlxI6g2PNaWsHoiRryto,spotify:track:62rlxI6g2PNaWsHoiRryto,https://api.spotify.com/v1/tracks/62rlxI6g2PNa...,https://api.spotify.com/v1/audio-analysis/62rl...,217613,4


In [56]:
# now add names to each and the album that each track belongs to
first_df['Name'] = names_tracks

# lets also try to add the album names
first_df['Album Name'] = album_names

# add release date of album
first_df['Release Date'] = album_release_date

In [57]:
first_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,Name,Album Name,Release Date
0,0.735,0.444,10,-10.519,1,0.0684,0.20400,0.001200,0.1700,0.0984,...,audio_features,4g2c7NoTWAOSYDy44l9nub,spotify:track:4g2c7NoTWAOSYDy44l9nub,https://api.spotify.com/v1/tracks/4g2c7NoTWAOS...,https://api.spotify.com/v1/audio-analysis/4g2c...,202396,4,Lavender Haze,Midnights (3am Edition),2022-10-22
1,0.658,0.378,7,-8.300,1,0.0379,0.05930,0.000000,0.0976,0.0382,...,audio_features,199E1RRrVmVTQqBXih5qRC,spotify:track:199E1RRrVmVTQqBXih5qRC,https://api.spotify.com/v1/tracks/199E1RRrVmVT...,https://api.spotify.com/v1/audio-analysis/199E...,218271,4,Maroon,Midnights (3am Edition),2022-10-22
2,0.638,0.634,4,-6.582,1,0.0457,0.13300,0.000001,0.1520,0.5190,...,audio_features,02Zkkf2zMkwRGQjZ7T4p8f,spotify:track:02Zkkf2zMkwRGQjZ7T4p8f,https://api.spotify.com/v1/tracks/02Zkkf2zMkwR...,https://api.spotify.com/v1/audio-analysis/02Zk...,200690,4,Anti-Hero,Midnights (3am Edition),2022-10-22
3,0.659,0.323,9,-13.425,1,0.0436,0.73500,0.003210,0.1160,0.1540,...,audio_features,6ADDIJxxqzM9LMpm78yzQG,spotify:track:6ADDIJxxqzM9LMpm78yzQG,https://api.spotify.com/v1/tracks/6ADDIJxxqzM9...,https://api.spotify.com/v1/audio-analysis/6ADD...,256124,4,Snow On The Beach (feat. Lana Del Rey),Midnights (3am Edition),2022-10-22
4,0.694,0.380,2,-10.307,1,0.0614,0.41600,0.000008,0.1260,0.3760,...,audio_features,7gVWKBcfIW93YxNBi3ApIE,spotify:track:7gVWKBcfIW93YxNBi3ApIE,https://api.spotify.com/v1/tracks/7gVWKBcfIW93...,https://api.spotify.com/v1/audio-analysis/7gVW...,194207,4,"You're On Your Own, Kid",Midnights (3am Edition),2022-10-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,0.511,0.719,4,-3.457,1,0.0265,0.16600,0.000005,0.3400,0.2890,...,audio_features,0vvt4IZOMkRug195S4MUq0,spotify:track:0vvt4IZOMkRug195S4MUq0,https://api.spotify.com/v1/tracks/0vvt4IZOMkRu...,https://api.spotify.com/v1/audio-analysis/0vvt...,234547,4,If This Was A Movie,Speak Now (Deluxe Edition),2010-10-25
193,0.582,0.765,7,-3.648,1,0.0324,0.02680,0.000002,0.1030,0.5590,...,audio_features,2R7C9dDqv1UPycvepBFziA,spotify:track:2R7C9dDqv1UPycvepBFziA,https://api.spotify.com/v1/tracks/2R7C9dDqv1UP...,https://api.spotify.com/v1/audio-analysis/2R7C...,275960,4,Superman,Speak Now (Deluxe Edition),2010-10-25
194,0.541,0.453,2,-6.459,1,0.0266,0.74800,0.000000,0.2000,0.3770,...,audio_features,6IH2Z54gYPLqgJtxZjX80i,spotify:track:6IH2Z54gYPLqgJtxZjX80i,https://api.spotify.com/v1/tracks/6IH2Z54gYPLq...,https://api.spotify.com/v1/audio-analysis/6IH2...,292560,4,Back To December - Acoustic,Speak Now (Deluxe Edition),2010-10-25
195,0.563,0.443,5,-5.078,1,0.0247,0.84200,0.000000,0.2930,0.3130,...,audio_features,62rlxI6g2PNaWsHoiRryto,spotify:track:62rlxI6g2PNaWsHoiRryto,https://api.spotify.com/v1/tracks/62rlxI6g2PNa...,https://api.spotify.com/v1/audio-analysis/62rl...,217613,4,Haunted - Acoustic Version,Speak Now (Deluxe Edition),2010-10-25


In [58]:
# now we can cut out some features
# like urls, type, href

first_df.drop(['analysis_url', 'track_href', 'uri', 'type'], axis = 1)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,Name,Album Name,Release Date
0,0.735,0.444,10,-10.519,1,0.0684,0.20400,0.001200,0.1700,0.0984,97.038,4g2c7NoTWAOSYDy44l9nub,202396,4,Lavender Haze,Midnights (3am Edition),2022-10-22
1,0.658,0.378,7,-8.300,1,0.0379,0.05930,0.000000,0.0976,0.0382,108.034,199E1RRrVmVTQqBXih5qRC,218271,4,Maroon,Midnights (3am Edition),2022-10-22
2,0.638,0.634,4,-6.582,1,0.0457,0.13300,0.000001,0.1520,0.5190,96.953,02Zkkf2zMkwRGQjZ7T4p8f,200690,4,Anti-Hero,Midnights (3am Edition),2022-10-22
3,0.659,0.323,9,-13.425,1,0.0436,0.73500,0.003210,0.1160,0.1540,110.007,6ADDIJxxqzM9LMpm78yzQG,256124,4,Snow On The Beach (feat. Lana Del Rey),Midnights (3am Edition),2022-10-22
4,0.694,0.380,2,-10.307,1,0.0614,0.41600,0.000008,0.1260,0.3760,120.044,7gVWKBcfIW93YxNBi3ApIE,194207,4,"You're On Your Own, Kid",Midnights (3am Edition),2022-10-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,0.511,0.719,4,-3.457,1,0.0265,0.16600,0.000005,0.3400,0.2890,147.852,0vvt4IZOMkRug195S4MUq0,234547,4,If This Was A Movie,Speak Now (Deluxe Edition),2010-10-25
193,0.582,0.765,7,-3.648,1,0.0324,0.02680,0.000002,0.1030,0.5590,131.982,2R7C9dDqv1UPycvepBFziA,275960,4,Superman,Speak Now (Deluxe Edition),2010-10-25
194,0.541,0.453,2,-6.459,1,0.0266,0.74800,0.000000,0.2000,0.3770,141.757,6IH2Z54gYPLqgJtxZjX80i,292560,4,Back To December - Acoustic,Speak Now (Deluxe Edition),2010-10-25
195,0.563,0.443,5,-5.078,1,0.0247,0.84200,0.000000,0.2930,0.3130,80.962,62rlxI6g2PNaWsHoiRryto,217613,4,Haunted - Acoustic Version,Speak Now (Deluxe Edition),2010-10-25


In [43]:
#